In [26]:
import json
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# ✅ 1️⃣ 학습 데이터 로드
with open("labeling_novels_filtered.json", "r", encoding="utf-8") as f:
    labeled_data = json.load(f)

In [27]:
# ✅ 2️⃣ 데이터셋 변환 (프롬프트 추가)
train_data = [{
    "input": f"다음 문장 중 작가의 연혁과 수상이력은 제외하고 스토리부분만 추출 해주세요. 등장인물의 직업, 스토리 전개, 주요 사건을 포함해주세요. 스토리가 없을 경우 줄거리 없음으로 처리하세요.:\n\n{item['description']}",
    "target": item["summary"]
} for item in labeled_data]

dataset = Dataset.from_dict({
    "input": [item["input"] for item in train_data],
    "target": [item["target"] for item in train_data]
})

In [28]:
# ✅ 3️⃣ KoBART 모델 & 토크나이저 로드
model_name = "gogamza/kobart-summarization"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [16]:
# ✅ 4️⃣ 데이터 토크나이징 (Trainer가 이해할 수 있는 형태로 변환)
def preprocess_data(example):
    model_inputs = tokenizer(
        example["input"], 
        max_length=1024, 
        truncation=True, 
        padding="max_length"
    )
    
    # 🔹 "target"을 "labels"로 변환 후 토크나이징
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target"], 
            max_length=250, 
            truncation=True, 
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [17]:
# ✅ 5️⃣ 데이터셋 변환 (batched=True로 속도 최적화)
tokenized_dataset = dataset.map(preprocess_data, batched=True, remove_columns=["input", "target"])

Map:   0%|          | 0/218 [00:00<?, ? examples/s]

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
# ✅ 6️⃣ 학습 설정
training_args = TrainingArguments(
    output_dir="./kobart_summary_finetuned",
    evaluation_strategy="epoch",  # 🔹 매 epoch마다 평가
    save_strategy="epoch",        # 🔹 매 epoch마다 저장
    per_device_train_batch_size=2,  # 🔹 배치 크기 (GPU 메모리에 따라 조정 가능)
    per_device_eval_batch_size=2,
    num_train_epochs=5,  # 🔹 학습 횟수 (필요에 따라 조정 가능)
    weight_decay=0.01,  # 🔹 가중치 감쇠 (과적합 방지)
    save_total_limit=5,  # 🔹 체크포인트 최대 개수
    logging_dir="./logs",
    logging_steps=100
)

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# ✅ 7️⃣ 🤗 Datasets의 train_test_split() 사용
dataset_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# ✅ 8️⃣ train/eval 데이터셋 설정
train_data = dataset_split["train"]
eval_data = dataset_split["test"]

# ✅ 9️⃣ Trainer 객체 생성 (eval_dataset 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,  # 🔹 평가 데이터셋 추가!
    tokenizer=tokenizer
)


/var/folders/wz/cf10k4857715p70flsxw2g9m0000gn/T/ipykernel_70500/3899536195.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# ✅ 10 학습 시작 🚀
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.720338
2,1.985400,0.673293
3,0.539300,0.692524
4,0.320900,0.722187
5,0.213200,0.735216


TrainOutput(global_step=435, training_loss=0.7166613732261219, metrics={'train_runtime': 192.9941, 'train_samples_per_second': 4.508, 'train_steps_per_second': 2.254, 'total_flos': 530470718668800.0, 'train_loss': 0.7166613732261219, 'epoch': 5.0})

In [21]:
# ✅ 9️⃣ 모델 저장
model.save_pretrained("./kobart_summary_finetuned")
tokenizer.save_pretrained("./kobart_summary_finetuned")

print("✅ KoBART 파인튜닝 완료! 모델이 './kobart_summary_finetuned'에 저장되었습니다.")

✅ KoBART 파인튜닝 완료! 모델이 './kobart_summary_finetuned'에 저장되었습니다.


In [22]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

best_checkpoint = "./kobart_summary_finetuned/checkpoint-174"  # epoch 2 모델 체크포인트
save_path = "./kobart_best_model"

# 모델과 토크나이저 불러오기
model = BartForConditionalGeneration.from_pretrained(best_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(best_checkpoint)

# 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ 가장 좋은 모델이 '{save_path}'에 저장되었습니다!")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ 가장 좋은 모델이 './kobart_best_model'에 저장되었습니다!


In [2]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# ✅ 저장된 모델 불러오기
best_checkpoint = "./kobart_best_model"
model = BartForConditionalGeneration.from_pretrained(best_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(best_checkpoint)

# ✅ 테스트할 입력 문장
test_text = "먼 바다에서 펼쳐지는 노인의 고독한 사투!\n\n미국 현대 문학의 개척자라 불리는 어니스트 헤밍웨이의 대표작 『노인과 바다』. 퓰리처상 수상작이자 헤밍웨이의 마지막 소설로, 작가 고유의 소설 수법과 실존 철학이 집약된 헤밍웨이 문학의 결정판이다. 한 노인의 실존적 투쟁과 불굴의 의지를 절제된 문장으로 강렬하게 그려냈다. 십여 년 동안 이렇다 할 작품을 내놓지 못했던 헤밍웨이는 이 작품을 통해 작가적 생명력을 재확인하고 삶을 긍정하는 성숙한 태도를 보여주었다. 개인주의와 허무주의를 넘어 인간과 자연을 긍정하고 진정한 연대의 가치를 역설한다. 감정을 절제한 문체와 사실주의 기법, 다양한 상징과 전지적 화법을 활용하여 작품의 깊이를 더했다."

# ✅ 입력 문장을 토큰화
input_ids = tokenizer(
    test_text, 
    return_tensors="pt", 
    max_length=1024, 
    truncation=True
    
    ).input_ids

# ✅ 모델로 요약 생성
summary_ids = model.generate(
    input_ids, 
    max_length=200, 
    num_beams=5, 
    early_stopping=True,
    repetition_penalty=1.5,
    length_penalty=1.5,
    temperature=0.8)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("🔹 원문: ", test_text)
print("🔹 요약 결과: ", summary_text)


🔹 원문:  먼 바다에서 펼쳐지는 노인의 고독한 사투!

미국 현대 문학의 개척자라 불리는 어니스트 헤밍웨이의 대표작 『노인과 바다』. 퓰리처상 수상작이자 헤밍웨이의 마지막 소설로, 작가 고유의 소설 수법과 실존 철학이 집약된 헤밍웨이 문학의 결정판이다. 한 노인의 실존적 투쟁과 불굴의 의지를 절제된 문장으로 강렬하게 그려냈다. 십여 년 동안 이렇다 할 작품을 내놓지 못했던 헤밍웨이는 이 작품을 통해 작가적 생명력을 재확인하고 삶을 긍정하는 성숙한 태도를 보여주었다. 개인주의와 허무주의를 넘어 인간과 자연을 긍정하고 진정한 연대의 가치를 역설한다. 감정을 절제한 문체와 사실주의 기법, 다양한 상징과 전지적 화법을 활용하여 작품의 깊이를 더했다.
🔹 요약 결과:  한 노인의 실존적 투쟁과 불굴의 의지를 절제된 문체로 강렬하게 그려낸 작품으로, 개인주의와 허무주의를 넘어 인간과 자연을 긍정하고 진정한 연대의 가치를 역설한다.


In [ ]:
### 
## summary "" 로 작업용 키값을 설정했더니 줄거리가 없다고 인식하는 경우가 많았음